# CS337 - OPERATING SYSTEMS- Project 6 Software Synchronization
#### By: Matthew Bass


**Project Overview:**

In the project_6 file you I start by implementing a simple version of the
race condition so I can test my synchronization solutions. In this file
 I will create well designed simulations to test mutual exclusion, progress,
 bounded wait-time, and N-threads when possible.

The tests for each solution:


<img src= "Users/matthewbass/Documents/School_Colby/Colby/spring22/CS337-Operating-Systems/Projects/Proj6/pics/test_reqs.png" >


To test the synchronization solutions the race conditions will be a global
counter that all the threads are incrementing. If the counter is the correct
number after all the threads are done working then we no the race condition
has been solved

A **race condition** is, a situation in which multiple threads or processes
read and write a shared data item and the final result depends on the
relative timing of their execution. So this is what we are going to be trying
 to prevent.


### PREREQUISITES:
    Python 3
    Jupyter
    jupyter_contrib_nbextensions
    Numpy
    threading
    datclasses
    autopep8


<br>

-----

### Setup:

To start I am using this code to make all my code blocks hide-able, thanks to
[stackoverflow](https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer).



In [21]:
%%HTML
<script>
    function luc21893_refresh_cell(cell) {
        if( cell.luc21893 ) return;
        cell.luc21893 = true;
        console.debug('New code cell found...' );

        var div = document.createElement('DIV');
        cell.parentNode.insertBefore( div, cell.nextSibling );
        div.style.textAlign = 'right';
        var a = document.createElement('A');
        div.appendChild(a);
        a.href='#'
        a.luc21893 = cell;
        a.setAttribute( 'onclick', "luc21893_toggle(this); return false;" );

        cell.style.visibility='hidden';
        cell.style.position='absolute';
        a.innerHTML = '[show code]';

    }
    function luc21893_refresh() {
        if( document.querySelector('.code_cell .input') == null ) {
            // it apeears that I am in a exported html
            // hide this code
            var codeCells = document.querySelectorAll('.jp-InputArea')
            codeCells[0].style.visibility = 'hidden';
            codeCells[0].style.position = 'absolute';
            for( var i = 1; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i].parentNode)
            }
            window.onload = luc21893_refresh;
        }
        else {
            // it apperas that I am in a jupyter editor
            var codeCells = document.querySelectorAll('.code_cell .input')
            for( var i = 0; i < codeCells.length; i++ ) {
                luc21893_refresh_cell(codeCells[i])
            }
            window.setTimeout( luc21893_refresh, 1000 )
        }
    }

    function luc21893_toggle(a) {
        if( a.luc21893.style.visibility=='hidden' ) {
            a.luc21893.style.visibility='visible';
            a.luc21893.style.position='';
            a.innerHTML = '[hide code]';
        }
        else {
            a.luc21893.style.visibility='hidden';
            a.luc21893.style.position='absolute';
            a.innerHTML = '[show code]';
        }
    }

    luc21893_refresh()
</script>

#### Imports:

In [30]:
import threading
import time
import numpy as np
from dataclasses import fields, dataclass
from abc import ABC, abstractmethod


#### Base Solution Class:
Now I am going to create an abstract class that will be used to make all the synchronization
solutions.

In [23]:
# Abstract class for all the solutions
class SyncSolution(ABC):

    def __init__(self) -> None:
        # Name of the solution (lock)
        self.name = None

        # Number of threads
        self.thread_count = None

        return

    @abstractmethod
    def lock(self) -> None:
        '''
        This method will be used to lock the solution.
        '''

        pass

    @abstractmethod
    def unlock(self) -> None:
        '''
        This method will be used to unlock the solution.
        '''
        pass

#### Base Testing:
To test the different solutions I am going to write the single main function that will increment the
global counter x INC_AMTS times. It will run the function over the
NUM_THREADS specified and will use the solution specified. Also, it will run
tests for mutual exclusion, progress, bounded wait-time, and N-threads when specified.


##### Starting Global Counter Variable:

In [24]:
# Setting global var x to 0
x = 0



##### Increment Function:

In [25]:
def increment():
    '''
    This function will be used to increment the global variable x
    Returns:
    '''
    # Get the global counter x
    global x

    # Increment the global counter x
    x += 1


<br>

##### Testing Functions:
Now I will write the three different helper functions for `thread_task` that
will be used to test the different aspects of a good synchronization solution.
Those being, mutual exclusion, progress, bounded wait-time. (We will also
test some solutions to see if they work with N-threads but this is just done
by changing the `thread_count` variable).

<br>

###### Mutual Exclusion:
Mutual exclusion is the ability to ensure that only one thread is executing a
 given critical section of code at a time. In more formal terms from our
 notes it is "The requirement that when one process is in a critical section
 that accesses shared resources, no other process may be in a critical
 section that accesses any of those shared resources". In the context of this
  testing the critical section is the `increment` function. because to
  increment the thread must first read the value of the global variable x and
   then add 1 to it so it is possible for another thread to increment the
   counter x while the first thread is incrementing it. This can lead to inconsistent
    and unexpected results. If a synchronization solution is not able to
     ensure mutual exclusion then it is not really a solution at all.

To validate if a count is correct (and there for mutual exclusive, as long as
 the count is great enough) I wrote `check_result` below:

In [26]:
def check_result(result: int, inc_amts: list) -> bool:
    '''
    This function will check the result of the the x output for the main_tasks

    Args:
        result (int): The result of the x output from the main_task
        inc_amts (list): The list of increments that were done per thread

    Returns:
        bool: True if the result is correct, False otherwise
    '''
    if result == sum(inc_amts):
        return True
    else:
        return False

<br>

Then the helper function `thread_task` to check `me` is written below:

In [27]:
def thread_task_me(lock: SyncSolution, thread_id: int, inc_amt: int, debug:
bool = True) -> None:
    '''
    This function will be used to test for mutual exclusion.

    Args:
        lock (SyncSolution): The solution to test
        thread_id (int): The id of the thread
        inc_amt (int): The amount of times to increment the global counter x
        debug (bool): If True, will print the results of the test

    Returns:
        None
    '''
    # Lock the solution
    lock.lock(thread_id,debug)

    # Increment the global counter x
    for i in range(inc_amt):
        increment()

    if debug:
        print(f"Thread {thread_id} is unlocking")

    # Unlock the solution
    lock.unlock(thread_id,debug)

    if debug:
        print(f'Thread {thread_id} is done')

    return


<br>

###### Progress:
Progress means that only threads that need the shared resource should participate in the
decision to access it.  If no process is executing in its critical section
and some processes wish to enter their critical sections, then only those
processes that are not executing in their remainder section can participate
in deciding which will enter its critical section next, and this selection
cannot be postponed  indefinitely. Basically no dead-lock or live-lock can
occur.

*dead-lock:* A situation in which two or more processes are unable to proceed
 because each is waiting for one of the others to do something.

*live-lock:* A situation in which two or more processes continuously change their state in response to changes in the other process(es) without doing any useful work.

Below is the code I wrote to test for progress it locks and unlocks within
the loop to cause more thread switches and cause the threads to have to be
locked and unlocked multiple times. It also then has extra work at the end of
 each thread to have them working on data that is independent of the other.

In [28]:
def thread_task_prog(lock: SyncSolution, thread_id: int, inc_amt: int, debug:
bool = True) -> None:
    '''
    This function will be used to test for progress.

    Args:
        lock (SyncSolution): The solution to test
        thread_id (int): The id of the thread
        inc_amt (int): The amount of times to increment the global counter x
        debug (bool): If True, will print the results of the test

    Returns:
        None
    '''

    # Increment the global counter x
    for i in range(inc_amt):
        # Lock the solution
        lock.lock(thread_id,debug)

        increment()

        if debug:
            print(f"Thread {thread_id} is unlocking")

        # Unlock the solution
        lock.unlock(thread_id,debug)

    prog_check = 0
    for _ in range(inc_amt * 10):
        prog_check += 1

    if debug:
        print(f'Thread {thread_id} is done')

    return

<br>

###### Bounded Wait Time (BWT):
Bounded wait time basically means that a ready thread should not wait forever.
In more formal terms it means, there exists a bound, or limit, on the number
of times other processes are allowed to enter their critical sections after a
 process has made request to enter its critical section and before that
 request is granted. The purpose of this condition is to make sure that every
  process gets the chance to actually enter its critical section so that no
  process starves forever. (Preventing starvation)


*starvation:* A situation in which a runnable process is overlooked
indefinitely by the scheduler; although it is able to proceed, it is never chosen.

To test for BWT I used the same time as the progress test but instead of
calling the normal `lock()` function I used the `lockSleep()` function which
uses the function `time.sleep(0.001)` to force contex switched while the
thread is locking.

Here is the helper function `thread_task_bwt()` below:

In [29]:
def thread_task_bwt(lock: SyncSolution, thread_id: int, inc_amt: int, debug:
bool = True) -> None:
    '''
    This function will be used to test for progress.

    Args:
        lock (SyncSolution): The solution to test
        thread_id (int): The id of the thread
        inc_amt (int): The amount of times to increment the global counter x
        debug (bool): If True, will print the results of the test

    Returns:
        None
    '''

    # Increment the global counter x
    for i in range(inc_amt):
        # Lock the solution
        lock.lockSleep(thread_id,debug)

        increment()

        if debug:
            print(f"Thread {thread_id} is unlocking")

        # Unlock the solution
        lock.unlock(thread_id,debug)

    prog_check = 0
    for _ in range(inc_amt * 10):
        prog_check += 1

    if debug:
        print(f'Thread {thread_id} is done')

    return

##### Thread Task Functions:
Now I am going to make the main thread task function which will increment the
global counter x, inc_amt times. It will run the function over the
NUM_THREADS specified and will use the solution specified. Also, it will run
tests for mutual exclusion, progress, bounded wait-time, and N-threads when specified.

In [ ]:
def thread_task(lock: SyncSolution = None, thread_id: int = 0, inc_amt: int =
100000, test:
str = "me",
                debug: bool = True):
    '''
    This is the thread task that will be used to test the software
    synchronization solutions.

    Args:
        lock (SyncSolution): The lock that will be used to synchronize
        thread_id (int): The number that will be used to synchronize (the thread number)
        inc_amt (int): The amount that will be used to increment the global variable x
        test (str): The test that we will be doing to determine what code to run
        debug (bool): If the debug flag is set to True, then the debug
    '''

    # If the SyncSolution is not specified, then we will use the default
    if lock is None:
         # If there is no lock, then just increment the global x variable
        for _ in range(inc_amt):
            increment()

        if debug:
            print(f'Thread {thread_id} is done')
        return

    # If the test is me, then we will run the main_task
    if test == "me":
        thread_task_me(lock, thread_id, inc_amt, debug)

    # If not mutual exclusion but is progress check do more thread switching
    # (and locked and unlocking) for each increment so they have to rely on
    # each other .
    elif test == "prog":
        thread_task_prog(lock, thread_id, inc_amt, debug)

    # If testing for bounded wait time use the lockSleep method which is the
    # same as the lock method but induces contex switches by calling
    # time.sleep(0.0001)
    elif test == "bwt":
        thread_task_bwt(lock, thread_id, inc_amt, debug)

    return

##### Simulation Functions:

Now I am going to write the functions for the simulation

To start I am going to write the `single_sim()` function which runs the thread
count simulation once.

In [ ]:
def single_sim(solution: SyncSolution = None,
               inc_amt: int = 100000,
               thread_count = 2 ,
               thread_multiplier = None,
               test: str = "me",
               debug: bool = False) -> int:
    '''
    This is the single_simulation that will be used to test the
    software synchronization solutions.

    Args:
        solution (SyncSolution): The lock that will be used to synchronize
        inc_amt (int): The amount that will be used to increment the global variable x
        thread_count (int): The number of threads that will be used
        thread_multiplier (list): The multiplier of the increment amount for each thread
        test (str): The test that we will be doing to determine what code to run
        debug (bool): If the debug flag is set to True, then the debug

    Returns:
        int: The result of the x output from the main_task

    '''

    global x
    x = 0

    # Create threads based on the info
    # create a lock
    lock = solution(thread_count)

    # Create threads
    threads = []
    for thread_num in range(1, thread_count + 1):
        thread = threading.Thread(target=thread_task,
                                  args=(
                                  lock, thread_num, inc_amts[thread_num - 1],
                                  debug))
        threads.append(thread)

    # Start threads
    for thread in threads:
        thread.start()

    # Wait for threads to finish
    for thread in threads:
        thread.join()

    return x

<br>

Before I write the function to run the simulation multiple times I am going
to write a helper function, `check_results()` to check the results of each
simulation.

In [ ]:
def check_results(results: list) -> bool:
    '''
    This function will check the results of the the x outputs for the main_tasks

    Args:
        results (list): The list of results from the main_tasks

    Returns:
        bool: True if the results are correct, False otherwise
    '''
    for result in results:
        if not check_result(result["x"]):
            return False
    return True


<br>

Now I am going to write the `main_simulations()` function which will run the
simulation for the times specified, along with all the other arguments that
there are for a single simulation.

In [ ]:
def main_simulations(solution: SyncSolution = None,
                     inc_amt: int = 100000,
                     thread_count = 2 ,
                     thread_multiplier = None,
                     test: str = "me",
                     debug: bool = False,
                     times: int = 1) -> None:
    """
    This is the main function that will be used to test the
    software synchronization solutions.r

    Args:
        solution (SyncSolution): The lock that will be used to synchronize
        times (int): The number of times that the simulation will be run
        inc_amt (int): The amount that will be used to increment the global variable x
        thread_count (int): The number of threads that will be used
        thread_multiplier (list): The multiplier of the increment amount for each thread
        test (str): The test that we will be doing to determine what code to run
        debug (bool): If the debug flag is set to True, then the debug



    Returns:
        None:

    """

    global x

     # Check the thread_multiplier
    if thread_multiplier is None:
        thread_multiplier = [1] * thread_count
    elif len(thread_multiplier) != thread_count:
        raise ValueError(f"The thread_multiplier must be the same length as the thread_count")

    inc_amts = [inc_amt * x for x in thread_multiplier]


    # Run the simulation times
    simulation_results = []
    for i in range(times):
        # Run the main task
        single_sim(solution, inc_amt, thread_count, thread_multiplier, test, debug)



        # Append the results to the list
        simulation_results.append({'iteration': i, 'x': x})

    # Check the results
    if check_results(simulation_results):
        print('\nAll results are correct!')

    # Else print the results that are incorrect
    else:
        print('\nThe results are incorrect!')

    # Print what the correct result should be
    print(f"The correct result should be {sum(inc_amts)}")

    # Print the results
    print('\nThe results are:')
    for result in simulation_results:
        print(f'Iteration {result["iteration"]+1}: x = {result["x"]}')

    return

<br>

#### Testing Simulation With No Synchronization:

Now that all the base code is done time to test it all out by trying to run
the simulation with no form of synchronization. A race condition should occur
 with the count not being correct (showing it isn't mutually exclusive at all)


#### Resources:
- [Petersons Solution Wiki](https://en.wikipedia.org/wiki/Peterson%27s_algorithm#Filter_algorithm:_Peterson's_algorithm_for_more_than_two_processes)
- [University of Idaho Notes](http://www2.cs.uidaho.edu/~krings/CS240/Notes.F13/240-13-12.pdf)
- [Stack Overflow Progress and BWT](https://stackoverflow.com/questions/33143779/what-is-progress-and-bounded-waiting-in-critical-section)